In [ ]:
# Pre-configured async. PPO launcher
# for Atari Gym Environment.
#
# Point tensorboard to User/tmp/ppo_test_gym_ppo
#
# Note: this one needs finetuning.
# 
# Paper: https://arxiv.org/pdf/1707.06347.pdf

In [ ]:
import os
from btgym.algorithms import AtariRescale42x42, Launcher, PPO, AacPolicy

In [ ]:

cluster_config=dict(
    host='127.0.0.1',
    port=12230,
    num_workers=6,  # Set according CPU's available 
    num_ps=1,
    log_dir=os.path.expanduser('~/tmp/ppo_test_gym_ppo'),
)

env_config = dict(
    class_ref=AtariRescale42x42,  # Gym env. preprocessed to normalized grayscale 42x42 pix.
    kwargs={'gym_id': 'Breakout-v0'}
)

policy_config = dict(
    class_ref=AacPolicy,
    kwargs={}
)

trainer_config=dict(
    class_ref=PPO,
    kwargs=dict(
        opt_learn_rate=[2e-4, 1e-4],  # Random log-uniform
        opt_end_learn_rate=1e-6,
        opt_decay_steps=20*10**6,
        model_gae_lambda=0.95,
        model_beta=[0.02, 0.001],  # Entropy reg, random log-uniform
        pi_old_update_period=10,  # = every 10'th train step
        num_epochs=3,  # PPO specific: mum. of SGD runs for every train step
        rollout_length=20,
        model_summary_freq=100,
        episode_summary_freq=10,
        env_render_freq=100,
    )
)

launcher = Launcher(
    cluster_config=cluster_config,
    env_config=env_config,
    trainer_config=trainer_config,
    policy_config=policy_config,
    test_mode=True,
    max_train_steps=50*10**6,
    root_random_seed=0,
    verbose=1
)

In [ ]:
launcher.run()